In [1]:
import  pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from tqdm import tqdm ,tqdm_notebook

In [2]:
import xgboost as xg
import sklearn.tree as tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.metrics import precision_recall_curve,precision_score,accuracy_score,roc_auc_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
sns.set(rc={'figure.figsize':(15,7)})

In [5]:
train=pd.read_csv('rpoj/train.csv')
test=pd.read_csv('rpoj/test.csv')
subm=pd.read_csv('rpoj/submission_example.csv')

In [6]:
train.head()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
0,0,35.0,0.0,2150.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,0
1,1,52.0,2.0,1250.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,57.0,1
2,2,29.0,3.0,1750.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0,0
3,3,33.0,3.0,1050.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,1
4,4,46.0,3.0,2250.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0,0


In [7]:
def pipl(df1):
    df=df1.copy()
    df['count_subj']=df['physics']+df['biology']+df['chemistry']+df['english']+df['geography']+df['history']
#     df['m_lesson_price']=MinMaxScaler(df['lesson_price'])
#     df['m_mean_exam_points']=MinMaxScaler(df['mean_exam_points'])
    submis=pd.DataFrame(df['Id'])
    df=df.drop(['Id','physics','biology','chemistry','english','geography','history'],axis=1)
    
#     df=df.drop(['Id','qualification','age','lesson_price','mean_exam_points','physics','biology','chemistry','english','geography','history'],axis=1)
    df=pd.get_dummies(df)
    return df,submis

In [8]:
df,submis=train.pipe(pipl)
df.head()

,age,years_of_experience,lesson_price,qualification,mean_exam_points,choose,count_subj
0,35.0,0.0,2150.0,2.0,74.0,0,0.0
1,52.0,2.0,1250.0,2.0,57.0,1,3.0
2,29.0,3.0,1750.0,1.0,66.0,0,1.0
3,33.0,3.0,1050.0,1.0,66.0,1,0.0
4,46.0,3.0,2250.0,2.0,73.0,0,1.0


In [9]:
X=df.drop('choose',axis=1)
y=df.choose

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=31)

In [11]:
def grad_bost(X_train,y_train,X_test,y_test,max_d,n_est,l_rate):
    xgb = xg.XGBClassifier(n_estimators=n_est, max_depth = max_d, learning_rate=l_rate, random_state=42)
    xgb.fit(X_train,y_train)
    xgb_y_pred=xgb.predict(X_test)
    rcf=roc_auc_score(y_test,xgb_y_pred)
#     df_ap=pd.DataFrame({'max_depth':[max_d],'n_estimators' :[n_est],'learning_rate' : [l_rate], 'roc_auc_score': [rcf]})
#     df_info=df_info.append(df_ap)
    return rcf


In [68]:
for md in tqdm_notebook(range(1,50)):
    for l in tqdm_notebook(np.linspace(0.1,1,10)):
        x=int(n//1)
        rc=grad_bost(X_train,y_train,X_test,y_test,md,700,l)
        df_ap=pd.DataFrame({'max_depth':[md],'learning_rate' : [l], 'roc_auc_score': [rc]})
#             print(df_ap.max_depth.values)
        score_df=score_df.append(df_ap)

In [67]:
score_df=pd.DataFrame()

In [63]:
for md in tqdm_notebook(range(1,10)):
    for l in tqdm_notebook(np.linspace(0.1,1,5)):
        for n in tqdm_notebook(np.linspace(500,2000,5)):
            x=int(n//1)
            rc=grad_bost(X_train,y_train,X_test,y_test,md,x,l)
            df_ap=pd.DataFrame({'max_depth':[md],'n_estimators' :[x],'learning_rate' : [l], 'roc_auc_score': [rc]})
#             print(df_ap.max_depth.values)
            score_df=score_df.append(df_ap)
      
            

In [69]:
score_df.sort_values('roc_auc_score')

,max_depth,learning_rate,roc_auc_score
0,1,0.1,0.566508
0,1,0.2,0.585197
0,9,0.8,0.587295
0,16,0.6,0.587499
0,14,1.0,0.587648
...,...,...,...
0,26,0.3,0.611714
0,31,1.0,0.612252
0,9,0.7,0.612437
0,6,1.0,0.612771


ПРогон разных моделей на сырых данных

In [12]:
def models(X_train,y_train,X_test,y_test):
    
# логистическая регрессия
    log_reg = LogisticRegression(solver = 'lbfgs', random_state = 42)
    log_reg.fit(X_train,y_train)
    log_y_predt=log_reg.predict(X_test)
    
    
# дерево с гиперпарамметрами    
    params={'max_depth':range(2,10),
    'min_samples_split':range(3,10),
    'min_samples_leaf':range(3,10)}
    
    
    clf=tree.DecisionTreeClassifier(criterion='entropy')
    grid=GridSearchCV(clf,param_grid=params,cv=9,scoring='roc_auc')
    grid.fit(X_train,y_train)
    b_clf=grid.best_estimator_
    b_clf_y_pred=b_clf.predict(X_test)
    
# случайный лес
    rcld=RandomForestClassifier(criterion='entropy' , random_state=42)
    grid2=GridSearchCV(rcld,param_grid=params,cv=10,n_jobs=1,scoring='roc_auc')
    grid2.fit(X_train,y_train)
    brclf=grid2.best_estimator_
    brclf_y_pred=brclf.predict(X_test)
    
    
    
# прогон адского бустинга
    cksf_ad=tree.DecisionTreeClassifier(max_depth=2,criterion='entropy')
    ad_cr=AdaBoostClassifier(cksf_ad,n_estimators=750,learning_rate=0.27,random_state=42)
    ad_cr.fit(X_train,y_train)
    ad_cr_y_pred=ad_cr.predict(X_test)
    
    
# прогон градиентного бустинга    
    xgb = xg.XGBClassifier(n_estimators=1000, max_depth = 2, learning_rate=0.2, reg_lambda=0.22, gamma=0.2, random_state=42)
    xgb.fit(X_train,y_train)
    xgb_y_pred=xgb.predict(X_test)
    
    
    print(f'Лог_регрессия_roc_auc_score: {roc_auc_score(y_test,log_y_predt)}')
    print(f'Дерево_с_гип__roc_auc_score: {roc_auc_score(y_test,b_clf_y_pred)}')
    print(f'Случ_лес_roc_auc_score: {roc_auc_score(y_test,brclf_y_pred)}')
    print(f'Адски_буст_roc_auc_score: {roc_auc_score(y_test,ad_cr_y_pred)}')
    print(f'xg_boost_roc_auc_score: {roc_auc_score(y_test,xgb_y_pred)}')
    
    
    return log_reg,b_clf,brclf,ad_cr,xgb
    

In [13]:
%%time
model=models(X_train,y_train,X_test,y_test)

Лог_регрессия_roc_auc_score: 0.5833434288923887
Дерево_с_гип__roc_auc_score: 0.5518049024034773
Случ_лес_roc_auc_score: 0.5287576588582107
Адски_буст_roc_auc_score: 0.6006986126866302
xg_boost_roc_auc_score: 0.5853460206977317
Wall time: 7min 16s


In [190]:
daf=pd.DataFrame({'dc_colums':X_train.columns,'feat_imp':model[2].feature_importances_})
# model[2].feature_importances_

In [191]:
daf.sort_values('feat_imp',ascending=False)

,dc_colums,feat_imp
0,lesson_price,0.537736
1,mean_exam_points,0.454915
2,golden_feat,0.007349


In [110]:
params={'max_depth':range(1,20),
   'min_samples_split':range(2,20),
   'min_samples_leaf':range(1,20)}
   
   

In [111]:
clf=tree.DecisionTreeClassifier(criterion='entropy')
grid=GridSearchCV(clf,param_grid=params,cv=9,scoring='roc_auc')

In [113]:
%%time
grid.fit(X_train,y_train)

Wall time: 22min 52s


GridSearchCV(cv=9, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='entropy',
                                              max_depth=None, max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': range(1, 20),
                         'min_samples_leaf': range(1, 20),
                      

In [116]:
b_clf=grid.best_estimator_

In [117]:
grid.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=18, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [118]:
b_clf_y_pred=b_clf.predict(X_test)

In [120]:
print(classification_report(y_test,b_clf_y_pred))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      2969
           1       0.46      0.08      0.14       331

    accuracy                           0.90      3300
   macro avg       0.68      0.54      0.54      3300
weighted avg       0.86      0.90      0.86      3300



In [126]:
df1,submis1=test.pipe(pipl)
df1.head()

,lesson_price,mean_exam_points,count_subj
0,2700.0,90.0,0.0
1,1800.0,71.0,2.0
2,1200.0,45.0,0.0
3,2950.0,92.0,1.0
4,1400.0,58.0,1.0


In [127]:
pred_answers=b_clf.predict(df1)

In [128]:
sub=pd.DataFrame(pred_answers,columns={'choose':pred_answers})
submision=pd.concat([submis1,sub],axis=1,names=['Id','choose'])

submision.to_csv('submission.csv',sep=',',index=False)

In [49]:
def ad_bost(X_train,y_train,X_test,y_test,n_est,l_rate):
    cksf_ad=tree.DecisionTreeClassifier(max_depth=2,criterion='entropy')
    ad_cr=AdaBoostClassifier(cksf_ad,n_estimators=n_est,learning_rate=l_rate,random_state=42)
    ad_cr.fit(X_train,y_train)
    ad_cr_y_pred=ad_cr.predict(X_test)
    rcf=roc_auc_score(y_test,ad_cr_y_pred)
    
    print(f'n_estimators :{n_est},learning_rate : {l_rate}, roc_auc_score: {rcf}')
    

In [51]:
def grad_bost(X_train,y_train,X_test,y_test,n_est,l_rate):
    xgb = xg.XGBClassifier(n_estimators=n_est, max_depth = 3, learning_rate=l_rate, reg_lambda=0.22, gamma=0.2, random_state=42)
    xgb.fit(X_train,y_train)
    xgb_y_pred=xgb.predict(X_test)
    rcf=roc_auc_score(y_test,xgb_y_pred)
    print(f'n_estimators :{n_est},learning_rate : {l_rate}, roc_auc_score: {rcf}')


In [58]:
for l in tqdm_notebook(np.linspace(0.1,0.5,8)):
    for n in np.linspace(300,1000,7):
        x=int(n//1)
        grad_bost(X_train,y_train,X_test,y_test,x,l)

n_estimators :300,learning_rate : 0.1, roc_auc_score: 0.6047043009384994
n_estimators :416,learning_rate : 0.1, roc_auc_score: 0.6060413802647497
n_estimators :533,learning_rate : 0.1, roc_auc_score: 0.6057045665227493
n_estimators :650,learning_rate : 0.1, roc_auc_score: 0.6057045665227493
n_estimators :766,learning_rate : 0.1, roc_auc_score: 0.6057045665227493
n_estimators :883,learning_rate : 0.1, roc_auc_score: 0.6057045665227493
n_estimators :1000,learning_rate : 0.1, roc_auc_score: 0.6057045665227493
n_estimators :300,learning_rate : 0.15714285714285714, roc_auc_score: 0.6130839419215072
n_estimators :416,learning_rate : 0.15714285714285714, roc_auc_score: 0.6171104433628869
n_estimators :533,learning_rate : 0.15714285714285714, roc_auc_score: 0.6171104433628869
n_estimators :650,learning_rate : 0.15714285714285714, roc_auc_score: 0.6171104433628869
n_estimators :766,learning_rate : 0.15714285714285714, roc_auc_score: 0.6171104433628869
n_estimators :883,learning_rate : 0.1571428

In [60]:
# n_estimators :1000,learning_rate : 0.8, roc_auc_score: 0.7846618357487922
# n_estimators :750,learning_rate : 0.27142857142857146, roc_auc_score: 0.8021739130434783
for l in tqdm_notebook(np.linspace(0.2,0.7,8)):
    for n in np.linspace(500,1000,7):
        x=int(n//1)
        ad_bost(X_train,y_train,X_test,y_test,x,l)

n_estimators :500,learning_rate : 0.2, roc_auc_score: 0.6093942542221281
n_estimators :583,learning_rate : 0.2, roc_auc_score: 0.6140943831475092
n_estimators :666,learning_rate : 0.2, roc_auc_score: 0.6068732389779994
n_estimators :750,learning_rate : 0.2, roc_auc_score: 0.6097259801432527
n_estimators :833,learning_rate : 0.2, roc_auc_score: 0.6122419075665055
n_estimators :916,learning_rate : 0.2, roc_auc_score: 0.6144210212477575
n_estimators :1000,learning_rate : 0.2, roc_auc_score: 0.612068412874629
n_estimators :500,learning_rate : 0.27142857142857146, roc_auc_score: 0.6078785923831251
n_estimators :583,learning_rate : 0.27142857142857146, roc_auc_score: 0.6063680183649983
n_estimators :666,learning_rate : 0.27142857142857146, roc_auc_score: 0.6040154099918698
n_estimators :750,learning_rate : 0.27142857142857146, roc_auc_score: 0.6053524893181201
n_estimators :833,learning_rate : 0.27142857142857146, roc_auc_score: 0.6071998770782477
n_estimators :916,learning_rate : 0.27142857

In [125]:
df1,submis1=test.pipe(pipl)
df1.head()

,lesson_price,mean_exam_points,count_subj
0,2700.0,90.0,0.0
1,1800.0,71.0,2.0
2,1200.0,45.0,0.0
3,2950.0,92.0,1.0
4,1400.0,58.0,1.0


***Лог_регрессия***

In [31]:
y_pred_subm_log=model[0].predict(df1)

In [ ]:
sub=pd.DataFrame(y_pred_subm_log,columns={'Survived':y_pred_subm_log})
submision=pd.concat([submis1,sub],axis=1,names=['PassengerId','Survived'])

submision.to_csv('gender_submission.csv',sep=',',index=False)

***Просто дерево с подбором гиперпарамметров***

In [100]:
y_pred_subm_tr=model[1].predict(df1)

In [ ]:
sub=pd.DataFrame(y_pred_subm_tr,columns={'Survived':y_pred_subm_tr})
submision=pd.concat([submis1,sub],axis=1,names=['PassengerId','Survived'])

submision.to_csv('gender_submission.csv',sep=',',index=False)

***Random forest classfier***

In [108]:
y_pred_subm_rfc=model[2].predict(df1)

In [26]:
sub=pd.DataFrame(y_pred_subm_rfc,columns={'Survived':y_pred_subm_rfc})
submision=pd.concat([submis1,sub],axis=1,names=['PassengerId','Survived'])

submision.to_csv('gender_submission.csv',sep=',',index=False)

,Pclass,Age,Fare,gender,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,7.8292,1,0,1,0
1,3,47.0,7.0000,0,0,0,1
2,2,62.0,9.6875,1,0,1,0
3,3,27.0,8.6625,1,0,0,1
4,3,22.0,12.2875,0,0,0,1


***ADA Boost***

In [ ]:
y_pred_subm_ad=model[3].predict(df1)

In [ ]:
sub=pd.DataFrame(y_pred_subm_ad,columns={'Survived':y_pred_subm_ad})
submision=pd.concat([submis1,sub],axis=1,names=['PassengerId','Survived'])

submision.to_csv('gender_submission.csv',sep=',',index=False)

***Градиентный бустинг***

In [38]:
y_pred_subm_xgd=model[4].predict(df1)

In [39]:
sub=pd.DataFrame(y_pred_subm_xgd,columns={'Survived':y_pred_subm_xgd})
submision=pd.concat([submis1,sub],axis=1,names=['PassengerId','Survived'])

submision.to_csv('gender_submission.csv',sep=',',index=False)